In [1]:
import breeze.linalg._
import breeze.numerics._
import breeze.stats._

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-5R90GJV:4041
SparkContext available as 'sc' (version = 3.4.0, master = local[*], app id = local-1696711172907)
SparkSession available as 'spark'


import breeze.linalg._
import breeze.numerics._
import breeze.stats._


In [2]:
import scala.util.Random

import scala.util.Random


In [3]:
class LinearRegression{
    def fit(x: DenseMatrix[Double], y: DenseVector[Double], 
            max_iter: Int, lr: Double): (DenseVector[Double], Double) = {
        
        var w = DenseVector.rand[Double](x.cols) * 0.001
        var b = 0.001
        
        for (_ <- 0 to max_iter) {
            val y_hat = (x * w) + b
            w :-= (lr* 2 * (x.t * (y_hat - y))).map(el => el / x.rows)
            b -= lr * 2 * sum(y_hat - y) / x.rows
        }
        (w, b)
    }
    
    def predict(x: DenseMatrix[Double], w: DenseVector[Double], b: Double): DenseVector[Double] ={
        val res = (x * w) + b
        res
    }
    
    def r2(y_pred: DenseVector[Double], y: DenseVector[Double]): Double = {
        val v = mean((y - mean(y)).map(el => pow(el, 2)))
        val u = mean((y - y_pred).map(el => pow(el, 2)))
        1.0 - u / v
    }
}

defined class LinearRegression


In [5]:
class LinearRegressionWithBatch{
    def fit(x: DenseMatrix[Double], y: DenseVector[Double], 
            max_iter: Int, lr: Double, batch_size: Int): (DenseVector[Double], Double) = {
        
        var w = DenseVector.rand[Double](x.cols) * 0.001
        var b = 0.001
        val random = new Random()
        
        for (_ <- 0 to max_iter) {
            for (j <- 0 to (x.rows/batch_size).toInt) {
                val mask = DenseVector.fill(batch_size)(random.nextInt(x.rows))
                val x_ = DenseMatrix(mask.map(i => x(i, 0 to 2).t.toArray).toArray:_*)
                val y_ = mask.map(i => y(i))

                val y_hat = (x_ * w) + b
                w :-= (lr* 2 * (x_.t * (y_hat - y_))).map(el => el / x_.rows)
                b -= lr * 2 * sum(y_hat - y_) / x_.rows

            }
        }
        (w, b)
    }
    
    def predict(x: DenseMatrix[Double], w: DenseVector[Double], b: Double): DenseVector[Double] ={
        val res = (x * w) + b
        res
    }
    
    def r2(y_pred: DenseVector[Double], y: DenseVector[Double]): Double = {
        val v = mean((y - mean(y)).map(el => pow(el, 2)))
        val u = mean((y - y_pred).map(el => pow(el, 2)))
        1.0 - u / v
    }
}

defined class LinearRegressionWithBatch


In [6]:
val model = new LinearRegressionWithBatch

val latent_model = DenseVector(1.5, 0.3, -0.7)
val x_train = DenseMatrix.rand(100000, 3)
val y_train= x_train * latent_model
val x_test = DenseMatrix.rand(10000, 3)
val y_test = x_test * latent_model

model: LinearRegressionWithBatch = LinearRegressionWithBatch@40977606
latent_model: breeze.linalg.DenseVector[Double] = DenseVector(1.5, 0.3, -0.7)
x_train: breeze.linalg.DenseMatrix[Double] =
0.21632137556526687    0.17217160105785823   0.8381024345001771
0.7249981758731348     0.7057396736826878    0.872404000292492
0.5007345858165266     0.7612067347895584    0.2970080033409366
0.964396660816061      0.10429887363593737   0.8824881773990643
0.5174252050925618     0.09145479888083852   0.44719526498689377
0.3663966894730113     0.3223392150221751    0.028323615048187367
0.7720807742130529     0.22422552283937636   0.7674803496825251
0.6954921875247653     0.025562139337242717  0.3241134814187665
0.035607517931438126   0.5765677256300001    0.6518110595078137
0.024164553971...


In [8]:
%%time
val (w, b) = model.fit(x_train, y_train, 10, 0.1, 128)

Time: 40.30966854095459 seconds.



w: breeze.linalg.DenseVector[Double] = DenseVector(1.4999999999999964, 0.3, -0.7000000000000007)
b: Double = 2.1559495148864146E-15


In [9]:
val y_pred = model.predict(x_test, w, b)

y_pred: breeze.linalg.DenseVector[Double] = DenseVector(0.4994297253401771, 0.31639343674761017, 0.4531807900428907, 0.004525894529398977, 0.939096621870869, 0.6983759553212124, 0.6333951728497987, 0.30187262239787677, -0.08480981729307827, 1.2791877243863916, 1.100826061165324, 1.1024833860556436, 0.872211751979992, -0.24599443200221308, 0.7947149654381519, 0.8741454391377234, -0.16178757769180924, 0.24667149851074066, 0.055298581480652635, 1.020915940156033, -0.1754134202554777, 1.0019827306641806, 0.07174439455003494, -0.2467613955309028, 0.8119735931905577, 0.4398245717086961, 0.4406438892395814, -0.023474723314043825, 1.3644768716141429, 0.3671767784587003, 0.23215922198592887, 0.7681822825371742, 0.4884364914219576, 0.9688807565305154, -0.07770238075851817, 0.9908422848765892, 1.1...


In [10]:
model.r2(y_pred, y_test)

res0: Double = 1.0
